In [1]:
import datetime
import pandas as pd
import forecastio
import getpass

In [2]:
# enter api
api_key = getpass.getpass()

········


In [3]:
len(api_key)

32

In [22]:
# enter coord
lat = 28.3852
lng = -81.5639

In [23]:
# start date - can only pull 2 years each day due to 1k free api calls limit
startYear = 2000
startDate = datetime.datetime(startYear,1,1)
ndays = (datetime.datetime(startYear+2,1,1) - datetime.datetime(startYear,1,1)).days

In [24]:
# empty hourly, daily df
dfh = pd.DataFrame()
dfd = pd.DataFrame()

# loop through historical days
for offset in range (0,ndays):
    # pull forecast
    fc = forecastio.load_forecast(api_key, lat, lng, time = startDate + datetime.timedelta(offset))
    
    # append hourly data to hourly df
    for i in range(0, len(fc.json['hourly'].get('data'))):
        df_i = pd.DataFrame([fc.json['hourly'].get('data')[i]])
        dfh = pd.concat([dfh, df_i], ignore_index=True)
    
    # append daily data to daily df
    for i in range(0, len(fc.json['daily'].get('data'))):
        df_i = pd.DataFrame([fc.json['daily'].get('data')[i]])
        dfd = pd.concat([dfd, df_i], ignore_index=True)

# unix time vars
hourlyTimeAttr = [col for col in dfh.columns if 'time' in col.lower()]
dailyTimeAttr = [col for col in dfd.columns if 'time' in col.lower()]

# convert unix time to datetime
for t in hourlyTimeAttr:
    dfh[t] = dfh[t].apply(lambda x: datetime.datetime.fromtimestamp(x) if(pd.notnull(x)) else None)
    
for t in dailyTimeAttr:
    dfd[t] = dfd[t].apply(lambda x: datetime.datetime.fromtimestamp(x) if(pd.notnull(x)) else None)

# export to csv
hourlyCSV = 'data\weather\weather_hourly_' + str(startYear) + '_' + str(startYear+1) + '.csv'
dailyCSV = 'data\weather\weather_daily_' + str(startYear) + '_' + str(startYear+1) + '.csv'
dfh.to_csv(hourlyCSV)
dfd.to_csv(dailyCSV)